In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
_seoul_sgg = './maps/output/seoul_sgg.geojson'
_seoul_sb = './maps/final/seoul_starbucks.geojson'

In [3]:
# 서울시 시군구 지도 데이터 불러오기
seoul_sgg_gdf = gpd.read_file(_seoul_sgg)
seoul_sgg_gdf.info()
seoul_sgg_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 9 columns):
SGG_CODE      25 non-null object
SGG_NM        25 non-null object
SEDAE         25 non-null int64
SEDAE_INGU    25 non-null float64
TOTAL_POP     25 non-null int64
MALE_POP      25 non-null int64
FEMALE_POP    25 non-null int64
65_OVER       25 non-null int64
geometry      25 non-null object
dtypes: float64(1), int64(5), object(3)
memory usage: 1.8+ KB


,SGG_CODE,SGG_NM,SEDAE,SEDAE_INGU,TOTAL_POP,MALE_POP,FEMALE_POP,65_OVER,geometry
0,11680,강남구,231612,54.29,561052,268941,292111,65060,"POLYGON ((127.1110354206454 37.48293504402488,..."
1,11740,강동구,177407,45.24,440359,218699,221660,56161,"POLYGON ((127.1458000886755 37.52198996684139,..."
2,11305,강북구,142533,29.67,328002,160252,167750,56530,"POLYGON ((127.0220295465139 37.6122870292313, ..."
3,11500,강서구,254257,47.24,608255,296175,312080,76032,"POLYGON ((126.8831662843492 37.55446522843518,..."
4,11620,관악구,255352,42.12,520929,262006,258923,70046,"POLYGON ((126.9704905925551 37.4753683309936, ..."


In [4]:
# 서울시 스타벅스 매장 지도 데이터 불러오기
seoul_sb_gdf = gpd.read_file(_seoul_sb)
seoul_sb_gdf.info()
seoul_sb_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 5 columns):
STORE_NAME    495 non-null object
STORE_TYPE    495 non-null object
SIG_CD        495 non-null object
SIG_KOR_NM    495 non-null object
geometry      495 non-null object
dtypes: object(5)
memory usage: 19.4+ KB


,STORE_NAME,STORE_TYPE,SIG_CD,SIG_KOR_NM,geometry
0,역삼아레나빌딩,general,11680,강남구,POINT (127.043069 37.501087)
1,논현역사거리,general,11680,강남구,POINT (127.022223 37.510178)
2,대치대원빌딩R,reserve,11680,강남구,POINT (127.062583 37.494668)
3,삼성역섬유센터R,reserve,11680,강남구,POINT (127.060651 37.50775)
4,압구정R,reserve,11680,강남구,POINT (127.033061 37.5273669)


In [5]:
# pandas를 이용해 서울시 시군구별 스타벅스 매장 개수 세기
starbucks_sgg_count = seoul_sb_gdf.pivot_table(
    index = 'SIG_CD', 
    values='SIG_KOR_NM', 
    aggfunc='count').rename(columns={'SIG_KOR_NM':'COUNT'})

starbucks_sgg_count.head()

,COUNT
SIG_CD,
11110,37
11140,50
11170,17
11200,9
11215,14


In [6]:
# pandas merge를 이용해 서울시 시군구 데이터에 스타벅스 매장 개수 컬럼 병합하기
seoul_sgg_final = pd.merge(
    seoul_sgg_gdf, 
    starbucks_sgg_count,
    how='left',
    left_on='SGG_CODE',
    right_on='SIG_CD'
)
seoul_sgg_final.head()

,SGG_CODE,SGG_NM,SEDAE,SEDAE_INGU,TOTAL_POP,MALE_POP,FEMALE_POP,65_OVER,geometry,COUNT
0,11680,강남구,231612,54.29,561052,268941,292111,65060,"POLYGON ((127.1110354206454 37.48293504402488,...",75
1,11740,강동구,177407,45.24,440359,218699,221660,56161,"POLYGON ((127.1458000886755 37.52198996684139,...",13
2,11305,강북구,142533,29.67,328002,160252,167750,56530,"POLYGON ((127.0220295465139 37.6122870292313, ...",5
3,11500,강서구,254257,47.24,608255,296175,312080,76032,"POLYGON ((126.8831662843492 37.55446522843518,...",14
4,11620,관악구,255352,42.12,520929,262006,258923,70046,"POLYGON ((126.9704905925551 37.4753683309936, ...",10


In [7]:
type(seoul_sgg_final)

geopandas.geodataframe.GeoDataFrame

In [8]:
# 서울시 시군구별 통계 지도 데이터 저장하기
seoul_sgg_final.to_file('./maps/final/seoul_sgg_stat.geojson', driver='GeoJSON')